In [1]:
##############################################################################
# Load a video file and create a prediction based on AI algorithm
##############################################################################

# This will be the routine inwhich we load a new video and then produce the classification for the
# complete video as a single prediction. Perhaps we should include the % confidence of the predicted class
# as well as the class label itself. 


In [2]:
# Initialize Core Packages

import cv2
import math
import sys
import argparse
import pandas as pd
from pandas import Series
import numpy as np
import datetime as dt
import shutil

# Helper libraries
import matplotlib.pyplot as plt

from skimage import io
from skimage import data_dir
from sklearn.preprocessing import LabelEncoder

import numpy as np
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join

# Deep Learning Framework 
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
from keras.optimizers import SGD
from keras.models import load_model
from keras.preprocessing import image

print(cv2.__version__)

C:\Users\derek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


3.4.2


In [3]:
# Declare Variables

videoFolder = "C:/Users/derek/Documents/Python_Scripts/Project_Adam/New_Video/"
videoFile = "C:/Users/derek/Documents/Python_Scripts/Project_Adam/New_Video/v_Trot_g09_c01.avi" #v_HorseRiding_g08_c06.avi or v_YoYo_g02_c02.avi
imagesFolder = "C:/Users/derek/Documents/Python_Scripts/Project_Adam/New_Video/Images"
algorithmFolder = "C:/Users/derek/Documents/Python_Scripts/Project_Adam/Model_Training_Routines/Classification/"

framerate_value = 30

# The shape of the individual image is 3 layers of 240 pixels by 320 pixels

horizontal_size = 240
vertical_size = 320
layer_count = 3

In [4]:
# Load old model weights and parameters

validation_features = np.load(algorithmFolder + "validation_features.npy")
validation_labels = np.load(algorithmFolder + "validation_labels.npy")
train_features = np.load(algorithmFolder + "train_features.npy")
train_labels = np.load(algorithmFolder + "train_labels.npy")
features_rnn = np.load(algorithmFolder + "features_rnn.npy")
labels_rnn = np.load(algorithmFolder + "labels_rnn.npy")
features_rnn_v = np.load(algorithmFolder + "features_rnn_v.np.npy")
labels_rnn_v = np.load(algorithmFolder + "labels_rnn_v.np.npy")


In [5]:
# This converts a video file into multiple images for scoring/prediction

vidcap = cv2.VideoCapture(videoFile)
count = 0
while True:
    #vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000)) # comment out for all frames
    success,image = vidcap.read()
    if not success:
        break
    cv2.imwrite(os.path.join(imagesFolder,"frame{:d}.jpg".format(count)), image)     # save frame as JPEG file
    count += 1 # 1 represents once per second; 5 for every 5 seconds
print("{} images are extacted in {}.".format(count,imagesFolder))

104 images are extacted in C:/Users/derek/Documents/Python_Scripts/Project_Adam/New_Video/Images.


In [6]:
# Import the Inception 3 algorithm 

from keras.applications import InceptionV3

In [7]:
# We will re-train the top layer of Inception with our own dataset. Thats why we define include_top to false
# The shape of the individual image is 3 layers of 240 pixels by 320 pixels

conv_base = InceptionV3(weights='imagenet',
                       include_top=False,
                       input_shape=(horizontal_size, vertical_size, layer_count)
                       )

In [8]:
# Load the previously trained algorithm

model = keras.models.load_model('C:/Users/derek/Documents/Python_Scripts/Project_Adam/Model_Training_Routines/Classification/modeltestDK.h5') # model.h5
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25166080  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 25,166,594
Trainable params: 25,166,594
Non-trainable params: 0
_________________________________________________________________


In [9]:
# compile the model - we must do this before running predictions

model.compile(optimizer=optimizers.adam(lr=2e-6),
             loss='categorical_crossentropy',
             metrics=['acc'])

In [10]:
# This function will loop through the target folder, bring all of the photos into the mix, and preprocess them. 

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.resize(img,(horizontal_size,vertical_size))
            # img = cv2.resize(img,(horizontal_size,vertical_size))
            img = np.reshape(img,[1,horizontal_size,vertical_size,layer_count])
            # img = np.reshape(img,[1, 6, 8, 2048])
            images.append(img)
    return images

In [11]:
# Execute the function and return all of the files

img2 = load_images_from_folder(imagesFolder)
# img2

[array([[[[215, 203, 197],
          [116, 104,  98],
          [202, 192, 182],
          ...,
          [186, 178, 165],
          [ 84,  76,  63],
          [168, 160, 147]],
 
         [[189, 181, 168],
          [ 14,   7,   1],
          [ 21,  22,  16],
          ...,
          [241, 238, 233],
          [240, 237, 232],
          [240, 237, 232]],
 
         [[241, 237, 232],
          [241, 237, 232],
          [240, 237, 232],
          ...,
          [226, 219, 210],
          [233, 226, 217],
          [241, 234, 225]],
 
         ...,
 
         [[179, 184, 193],
          [193, 198, 206],
          [176, 182, 187],
          ...,
          [151, 159, 166],
          [174, 182, 189],
          [157, 165, 172]],
 
         [[174, 181, 190],
          [159, 166, 175],
          [148, 156, 163],
          ...,
          [125, 132, 141],
          [133, 140, 149],
          [165, 172, 181]],
 
         [[122, 129, 138],
          [160, 167, 176],
          [178, 185, 194],
   

In [12]:
# stack up images list to pass for prediction

images = np.vstack(img2)

In [13]:
# Count the number of images

batch_size_pictures = len(img2)

In [14]:
# extracting features from the images using pretrained model
images = conv_base.predict(images)

In [15]:
# converting the images to 1-D form
images = images.reshape(batch_size_pictures, 6*8*2048)

In [16]:
# Create the prediction from our trained model

probability = model.predict(images, batch_size=batch_size_pictures) # batch =1
print(probability)

[[7.18654990e-02 9.28134561e-01]
 [7.51605257e-02 9.24839437e-01]
 [7.54470693e-06 9.99992490e-01]
 [1.60818686e-07 9.99999881e-01]
 [1.20420523e-07 9.99999881e-01]
 [1.96661199e-06 9.99998093e-01]
 [2.35138737e-06 9.99997616e-01]
 [3.21361609e-02 9.67863917e-01]
 [2.39453390e-02 9.76054668e-01]
 [6.00585900e-03 9.93994176e-01]
 [5.07197738e-01 4.92802292e-01]
 [5.22837304e-02 9.47716236e-01]
 [6.27669142e-05 9.99937177e-01]
 [3.30640189e-03 9.96693611e-01]
 [3.52850169e-01 6.47149920e-01]
 [2.90446371e-01 7.09553599e-01]
 [2.91659027e-01 7.08340943e-01]
 [2.38830909e-01 7.61169016e-01]
 [9.69260812e-01 3.07392608e-02]
 [7.59863973e-01 2.40136072e-01]
 [7.41572529e-02 9.25842822e-01]
 [7.79818511e-04 9.99220133e-01]
 [5.08332662e-02 9.49166715e-01]
 [1.15191888e-05 9.99988437e-01]
 [1.89844822e-03 9.98101532e-01]
 [7.47822523e-02 9.25217748e-01]
 [1.24668190e-02 9.87533212e-01]
 [3.92713770e-02 9.60728645e-01]
 [2.09645443e-02 9.79035497e-01]
 [7.33216584e-01 2.66783357e-01]
 [1.320712

In [17]:
# identify the maximum percentage value of the prediction

np.argmax(probability)  #0 is horse race, 1 is horse riding, 2 is yoyo

129

In [19]:
# Create a dataframe of the probabilities

PredictionResults = pd.DataFrame(probability, columns=['Canter','Trot'])
# PredictionResults = pd.DataFrame(classes)

print(PredictionResults)

           Canter      Trot
0    7.186550e-02  0.928135
1    7.516053e-02  0.924839
2    7.544707e-06  0.999992
3    1.608187e-07  1.000000
4    1.204205e-07  1.000000
5    1.966612e-06  0.999998
6    2.351387e-06  0.999998
7    3.213616e-02  0.967864
8    2.394534e-02  0.976055
9    6.005859e-03  0.993994
10   5.071977e-01  0.492802
11   5.228373e-02  0.947716
12   6.276691e-05  0.999937
13   3.306402e-03  0.996694
14   3.528502e-01  0.647150
15   2.904464e-01  0.709554
16   2.916590e-01  0.708341
17   2.388309e-01  0.761169
18   9.692608e-01  0.030739
19   7.598640e-01  0.240136
20   7.415725e-02  0.925843
21   7.798185e-04  0.999220
22   5.083327e-02  0.949167
23   1.151919e-05  0.999988
24   1.898448e-03  0.998102
25   7.478225e-02  0.925218
26   1.246682e-02  0.987533
27   3.927138e-02  0.960729
28   2.096454e-02  0.979035
29   7.332166e-01  0.266783
..            ...       ...
74   9.654110e-03  0.990346
75   9.281995e-03  0.990718
76   1.996961e-01  0.800304
77   6.759146e-01  0

In [20]:
# Create the prediction

classes = model.predict_classes(images, batch_size=1)
print(classes)

[1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [21]:
# Create a dataframe of the classes

PredictionClassResults = pd.DataFrame(classes, columns=['classification'])
# PredictionClassResults = pd.DataFrame(classes)

print(PredictionClassResults)

     classification
0                 1
1                 1
2                 1
3                 1
4                 1
5                 1
6                 1
7                 1
8                 1
9                 1
10                0
11                1
12                1
13                1
14                1
15                1
16                1
17                1
18                0
19                0
20                1
21                1
22                1
23                1
24                1
25                1
26                1
27                1
28                1
29                0
..              ...
74                1
75                1
76                1
77                0
78                0
79                0
80                1
81                1
82                1
83                0
84                0
85                1
86                1
87                1
88                1
89                1
90                1
91                1


In [22]:
# This will create a dataframe with all of the file names in the folder

filelist = os.listdir(imagesFolder)

# path = os.path.join(os.getcwd(),'C:/Users/derek/Documents/Python_Scripts/Project_Adam/New_Video/Images/')
# files = [os.path.join(path,i) for i in os.listdir(path) if os.path.isfile(os.path.join(path,i))]
# files

In [23]:
# make the file listing into a dataframe

filelisting = pd.DataFrame(filelist, columns=['FileName'])
# filelisting

In [24]:
# This will identify the video in the folder

videofilelist = [ f for f in listdir(videoFolder) if isfile(join(videoFolder,f)) ]
videofilelist

# path = os.path.join(os.getcwd(),videoFolder)
#files = [os.path.join(path,i) for i in os.listdir(path) if os.path.isfile(os.path.join(path,i))]
#files

['v_Trot_g09_c01.avi']

In [25]:
# add the video file names to the main prediction dataframe

PredictionResults.loc[:, 'VideoName'] = videofilelist
# PredictionResults

In [26]:
# add the image file names to the main prediction dataframe

PredictionResults['FileName'] = filelisting['FileName']
# PredictionResults

In [27]:
# add the file names to the main prediction dataframe

PredictionResults['classification'] = PredictionClassResults['classification']
# PredictionResults

In [28]:
# add the date to the main prediction dataframe

PredictionResults['DateTime'] = dt.datetime.now()
# PredictionResults

In [30]:
# Rearrange the Order of the Dataframe to look nicer.

PredictionResults = PredictionResults[['VideoName','FileName','DateTime','Canter','Trot','classification']]

#cols = PredictionResults.columns.tolist()
#cols = cols[-1:] + cols[:-1] 

In [31]:
# Export the dataframe into a csv file

export_csv = PredictionResults.to_csv(r'C:/Users/derek/Documents/Python_Scripts/Project_Adam/Model_Results/PredictionResults.csv',encoding='utf-8', index=False)

In [32]:
# Clean up and remove old files that will not be used

for the_file in os.listdir(imagesFolder):
    file_path = os.path.join(imagesFolder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)


In [33]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [34]:
# Load a single image to test

# img = cv2.imread('C:/Users/derek/Documents/Python_Scripts/Project_Adam/New_Video/Images/frame7.jpg') #frame7
# img = cv2.resize(img,(horizontal_size,vertical_size))
# img = np.reshape(img,[1,horizontal_size,vertical_size,layer_count])

In [35]:
# classes = model.predict(img)
# classes = model.predict_classes(img)

In [36]:
def get_accuracy(predictions, labels):
    """After predicting on each batch, check that batch's
    accuracy to make sure things are good to go. This is
    a simple accuracy metric, and so doesn't take confidence
    into account, which would be a better metric to use to
    compare changes in the model."""
    correct = 0
    for frame in predictions:
        # Get the highest confidence class.
        this_prediction = frame[0].tolist()
        this_label = frame[1]

        max_value = max(this_prediction)
        max_index = this_prediction.index(max_value)
        predicted_label = labels[max_index]

        # Now see if it matches.
        if predicted_label == this_label:
            correct += 1

    accuracy = correct / len(predictions)
    return accuracy